In [9]:
import pandas as pd
import psycopg2
from google.cloud import bigquery
from helper_fns import initial_cleanup

In [33]:
# client = bigquery.Client()
# client.query("SELECT id FROM `qtm-qmdatest.qm.quinnmurray` WHERE DATE(PARTITIONTIME) = '2020-09-24' LIMIT 10")

initial_query = "declare date_start date default \"2020-03-02\"; declare date_end date default \"2020-09-02\" ; SELECT session.id as sess_id, session.conn_rtt as rtt_ms, session.engaged, session.conversion_count, session.conversion_value, session.user.id as user, session.ts as start_time, session.locale.country_iso_code, session.device.name as device_name, session.device.browser_name as browser, session.os.name as os_name, ARRAY_LENGTH(session.hits) as hit_count, ARRAY_LENGTH(hits_column.events) as event_count FROM `qtm-qmdatest.qm.quinnmurray` as session,unnest(hits) as hits_column WHERE DATE(PARTITIONTIME) between date_start and date_end"

query


'declare date_start date default "2020-03-02"; declare date_end date default "2020-09-02" ; SELECT session.id as sess_id, session.conn_rtt as rtt_ms, session.engaged, session.conversion_count, session.conversion_value, session.user.id as user, session.ts as start_time, session.locale.country_iso_code, session.device.name as device_name, session.device.browser_name as browser, session.os.name as os_name, ARRAY_LENGTH(session.hits) as hit_count, ARRAY_LENGTH(hits_column.events) as event_count FROM `qtm-qmdatest.qm.quinnmurray` as session,unnest(hits) as hits_column WHERE DATE(PARTITIONTIME) between date_start and date_end'

In [11]:
# json = pd.read_json(r"C:\Users\Vincent\Documents\Personal\Brian\Work\experiments\quantum_metric_test\subset.json")
json = pd.read_json(r"C:\Users\Vincent\Documents\Personal\Brian\Work\experiments\quantum_metric_test\full_data_set.json")
json.head()

,id,cookie,ts,day,hour,last_updated,screen_width,screen_height,conversion_count,conversion_value,...,user,locale,device,os,landing_url,last_url,referrer,first_referrer,hits,PARTITIONTIME
0,4828495,757e0cd28bcfdd3c88ba89491d3ec682,1600922999,18529,444700,1600923006,1280,960,0,0,...,"{'id': '4816048', 'email': '', 'login': '', 'c...","{'ip': '77.88.5.40', 'country_iso_code': 'RU',...","{'name_id': '2', 'name': 'unknown', 'type_id':...","{'id': '8', 'name': '', 'type': 'unknown'}","{'id': '262', 'domain_id': '2', 'domain': 'qui...","{'id': None, 'domain_id': '2', 'domain': 'quin...","{'id': '1', 'category_id': '1', 'category': 'D...","{'id': '1', 'category_id': '1', 'category': 'D...","[{'id': '31166714', 'session_id': '4828495', '...",2020-09-24 00:00:00 UTC
1,4831185,9f4035bccca5f3fe11fcdd2d4661ee8c,1600958005,18529,444710,1600958162,3440,1440,0,0,...,"{'id': '2933276', 'email': '', 'login': '', 'c...","{'ip': '50.250.155.9', 'country_iso_code': 'US...","{'name_id': '4', 'name': 'Macintosh', 'type_id...","{'id': '12', 'name': 'macOS', 'type': 'macOS'}","{'id': '262', 'domain_id': '2', 'domain': 'qui...","{'id': None, 'domain_id': '2', 'domain': 'quin...","{'id': '1', 'category_id': '1', 'category': 'D...","{'id': '1', 'category_id': '1', 'category': 'D...","[{'id': '31185709', 'session_id': '4831185', '...",2020-09-24 00:00:00 UTC
2,4831330,9f4035bccca5f3fe11fcdd2d4661ee8c,1600960595,18529,444711,1600960805,3440,1440,0,0,...,"{'id': '2933276', 'email': '', 'login': '', 'c...","{'ip': '50.250.155.9', 'country_iso_code': 'US...","{'name_id': '4', 'name': 'Macintosh', 'type_id...","{'id': '12', 'name': 'macOS', 'type': 'macOS'}","{'id': '262', 'domain_id': '2', 'domain': 'qui...","{'id': None, 'domain_id': '2', 'domain': 'quin...","{'id': '1', 'category_id': '1', 'category': 'D...","{'id': '1', 'category_id': '1', 'category': 'D...","[{'id': '31186734', 'session_id': '4831330', '...",2020-09-24 00:00:00 UTC
3,4832203,1699c06091020c53fa148cb344644d6f,1600972953,18529,444714,1600973253,1440,900,0,0,...,"{'id': '4819573', 'email': '', 'login': '', 'c...","{'ip': '98.115.163.63', 'country_iso_code': 'U...","{'name_id': '4', 'name': 'Macintosh', 'type_id...","{'id': '13', 'name': 'Mac OS X', 'type': 'MacO...","{'id': '270', 'domain_id': '2', 'domain': 'qui...","{'id': None, 'domain_id': '2', 'domain': 'quin...","{'id': '1', 'category_id': '1', 'category': 'D...","{'id': '1', 'category_id': '1', 'category': 'D...","[{'id': '31192730', 'session_id': '4832203', '...",2020-09-24 00:00:00 UTC
4,4832042,5dd3871d583ca4942a04791207e3e199,1600970329,18529,444713,1600970723,1440,900,0,0,...,"{'id': '4819573', 'email': '', 'login': '', 'c...","{'ip': '98.115.163.63', 'country_iso_code': 'U...","{'name_id': '4', 'name': 'Macintosh', 'type_id...","{'id': '13', 'name': 'Mac OS X', 'type': 'MacO...","{'id': '270', 'domain_id': '2', 'domain': 'qui...","{'id': None, 'domain_id': '2', 'domain': 'quin...","{'id': '1', 'category_id': '1', 'category': 'D...","{'id': '1', 'category_id': '1', 'category': 'D...","[{'id': '31191619', 'session_id': '4832042', '...",2020-09-24 00:00:00 UTC


In [12]:
json.shape

(791, 32)

# Cleaning up some columns

In [3]:
df = initial_cleanup(json)